In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv('taac_assistant_taac.csv')

In [ ]:
df.head()

,TaskId,User_Search_Term,Ad,Website,Relevance
0,1,wwww ncquickpass com,Nc Quick Pass - Pay Your Bill Online,www.doxo.com/pay/nc-quick-pass,Other
1,2,peloton plano tx,Studio Cycle Comparison - Find The Best Exerci...,www.nordictrack.com/Studio-Cycles/S22i,Other
2,3,antelope canyon,Hotels near Antelope Canyon - 100% Real Custom...,www.booking.com/Antelope-Canyon/Hotels,Other
3,4,get vaccine after covid,Janssen COVID-19 Vaccine - Authorized For Emer...,www.janssencovid19vaccine.com,Other
4,5,ahs.com/my-accountlogin,Find First american home warranty login - Chec...,www.searchandshopping.org/Your Search/Results,Other


In [ ]:
#Converting the Relevance to numerical values to enable analysis
#Convert Relevance to numerical
mapping = {
    'Good' : 1,
    'Other' : 0,
}
df['Relevance'] = df['Relevance'].replace(mapping)
df.head()

,TaskId,User_Search_Term,Ad,Website,Relevance
0,1,wwww ncquickpass com,Nc Quick Pass - Pay Your Bill Online,www.doxo.com/pay/nc-quick-pass,0
1,2,peloton plano tx,Studio Cycle Comparison - Find The Best Exerci...,www.nordictrack.com/Studio-Cycles/S22i,0
2,3,antelope canyon,Hotels near Antelope Canyon - 100% Real Custom...,www.booking.com/Antelope-Canyon/Hotels,0
3,4,get vaccine after covid,Janssen COVID-19 Vaccine - Authorized For Emer...,www.janssencovid19vaccine.com,0
4,5,ahs.com/my-accountlogin,Find First american home warranty login - Chec...,www.searchandshopping.org/Your Search/Results,0


In [ ]:
## Get the Independent Features

X=df.drop('Relevance',axis=1)

In [ ]:
## Get the Dependent features
y=df['Relevance']

In [ ]:
y.value_counts()

1    8
0    7
Name: Relevance, dtype: int64

In [ ]:
X.shape

(15, 4)

In [ ]:
y.shape

(15,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=5000

Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['User_Search_Term'][1]

'peloton plano tx'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['User_Search_Term'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [ ]:
corpus

['wwww ncquickpass com',
 'peloton plano tx',
 'antelop canyon',
 'get vaccin covid',
 'ah com accountlogin',
 'nike',
 'cfl fixtur',
 'nationwid pet insur',
 'nike',
 'use car',
 'car rental lubbock tx',
 'augusta tech adn',
 'hampton inn guntersvil al',
 'white strip',
 'florist laguna beach']

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[191, 871, 3130],
 [2802, 1432, 1598],
 [1998, 2321],
 [1603, 3759, 4285],
 [1540, 3130, 4666],
 [987],
 [3247, 2540],
 [4982, 4349, 907],
 [987],
 [4387, 4535],
 [4535, 3705, 1147, 1598],
 [3101, 1964, 4436],
 [3626, 3598, 1616, 3874],
 [692, 3525],
 [1561, 4996, 4772]]

Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  191  871 3130]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 2802 1432 1598]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 1998 2321]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 1603 3759 4285]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 1540 3130 4666]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  987]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 3247 2540]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 4982 4349  907]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  987]
 [   0    0    0    0    0    0    0    0    0    0    

In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,  191,  871, 3130], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(15, (15,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((15, 20), (15,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=16)

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.6938 - accuracy: 0.4000 - val_loss: 0.6960 - val_accuracy: 0.4000
Epoch 2/10
1/1 [==============================] - 0s 39ms/step - loss: 0.6919 - accuracy: 0.5000 - val_loss: 0.6965 - val_accuracy: 0.4000
Epoch 3/10
1/1 [==============================] - 0s 39ms/step - loss: 0.6928 - accuracy: 0.5000 - val_loss: 0.6968 - val_accuracy: 0.4000
Epoch 4/10
1/1 [==============================] - 0s 40ms/step - loss: 0.6924 - accuracy: 0.5000 - val_loss: 0.6968 - val_accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 0s 41ms/step - loss: 0.6902 - accuracy: 0.6000 - val_loss: 0.6969 - val_accuracy: 0.4000
Epoch 6/10
1/1 [==============================] - 0s 41ms/step - loss: 0.6905 - accuracy: 0.5000 - val_loss: 0.6969 - val_accuracy: 0.4000
Epoch 7/10
1/1 [==============================] - 0s 38ms/step - loss: 0.6844 - accuracy: 0.5000 - val_loss: 0.6968 - val_accuracy: 0.4000
Epoch 8/10
1/1 [=============

Performance Metrics And Accuracy

In [ ]:
y_pred1 = np.argmax(model1.predict(X_test),axis=1)

1/1 [==============================] - 1s 531ms/step


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

array([[2, 0],
       [3, 0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.4

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.40      1.00      0.57         2
           1       0.00      0.00      0.00         3

    accuracy                           0.40         5
   macro avg       0.20      0.50      0.29         5
weighted avg       0.16      0.40      0.23         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model1.predict([["enterprise rent-a-car","Enterprise Rent-A-Car - When You're Ready, Were Ready - Enterprise Official Site","www.enterprise.com"]])

ValueError: ignored